# Chapter 2: Information Criterions and Bootstrap Confidence Intervals

In this chapter, we'll cover the Akaike Information Criterion (AIC) and Bayesian Information Criterion (BIC) for nested models, as well as the bootstrap method for confidence intervals. 

## Nested Models

Let's say we have model A with variables $\theta_1$, $\theta_2$ and $\theta_3$. Model B is nested in model A if model B contains a subset of the variables of model A, e.g. if model B has variables $\theta_1$ and $\theta_3$. We call model B a $\textbf{nested model}$. 
Model B is $\textbf{not}$ a nested model if it contains a variable that model A doesn't contain, e.g. if model B has variables $\theta_1$, $\theta_3$ and $\theta_4$. 

[overgang van nested models naar AIC and BIC en waarom we naar nested models kijken]


## Akaike Information Criterion (AIC)

tekst voor AIC en BIC

Zo krijg je een formule in de tekst
$AIC = 2*2=4$

Zo krijg je een formule in het midden. De tag is voor de nummering.
De unit boven [Eq. 1](#eq:AIC) maakt het mogelijk om er zo naar te refereren.


The Akaike Information Criterion (AIC) is an estimator of prediction error of statistical models. It is a method for evaluating how well a model fits the data it was generated from. 

It combines the max-liklihood and the number of parameters $p_k$.

<a id="eq:AIC"></a>
$$AIC = - 2\max\left[\ln(L)\right] + 2p_k,\tag{1}$$



## Bayesian Information Criterion (BIC)


## Bootstrap and confidence intervals

The bootstrap technique is used to estimate a confidence interval for a parameter which inherently does not have a confidence interval. First we need to understand what a confidence interval is. 

Confidence intervals

It is a way to easily say some value can be found with a certain confidence within an interval. For example, you want to find your parameter $\theta$ with 95% confidence. It would not suffice to find some value $\theta$ within your data and say that you are 95% confident that the found $\theta$ is the actual $\theta$. You need to found the boundary or the interval in which you find the $\theta$ 95% of the time. Thus eventually founding ($\theta_{lower}$, $\theta_{higher}$), now you have expressed an interval in which you can say a found $\theta$ resides 95% of the time.
